In [1]:
#Import Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
# Define the eBay filters dictionary
ebay_filters = {
    "item_conditions": {
        "New": 1000,
        "Open box": 1500,
        "Used": 3000,
        "Certified Refurbished": 2000,
        "Excellent - Refurbished": 2500,
        "Very Good": 3000,
        "Good": 4000,
        "For Parts or Not Working": 7000
    },
    "item_locations": {
        "Domestic": 1,
        "International": 2,
        "Continent": 3,
    },
    "directories": {
        "No Directory": 0,
        "Consumer Electronics": 9355,
        "Clothing, Shoes & Accessories": 11450,
        "Health & Beauty": 26395,
        "Home & Garden": 11700,
        "Sporting Goods": 382,
        "Toys & Hobbies": 220,
        "Books": 267,
        "Video Games & Consoles": 1249,
        "Collectibles": 1,
        "Business & Industrial": 12576,
        "Automotive": 6000, 
    },
    "categories": {
        "No Category": 0,
        "Cell Phones & Smartphones": 9355,
        "Laptops & Netbooks": 175673,
        "Watches": 31387,
        "Furniture": 3197,
        "Action Figures": 2605,
        "Jewelry & Watches": 281,
        "Cameras & Photo": 625,
        "Pet Supplies": 1281,
        "Crafts": 14339,
        "Computers/Tablets & Networking": 58058,
        "Cars & Trucks": 6001,  
        "Motorcycles": 6024,  
        "Car & Truck Parts": 6030,  
        "Motorcycle Parts": 10063,  
        "Automotive Tools & Supplies": 34998,  
    },
    "sort_order": {
        "Best Match": 12,
        "Time: ending soonest": 1,
        "Time: newly listed": 10,
        "Price + Shipping: lowest first": 15,
        "Price + Shipping: highest first": 16,
        "Distance: nearest first": 7
    }
}


In [5]:
# Define the base URL for the eBay search
url = "https://www.ebay.com/sch/i.html"

# Define the query parameters for the search request
params = {
    '_from': 'R40',
    '_nkw': 'iphone 13',
    'LH_ItemCondition': ebay_filters["item_conditions"]["Used"],  # Item condition; 'New'.
    'LH_PrefLoc': ebay_filters["item_locations"]["International"],
    '_udlo': '200',  # Minimum price.
    '_udhi': '400',  # Maximum price.
    '_dcat': ebay_filters["directories"]["No Directory"],  # Filter by directory ID; "Consumer Electronics".
    '_sacat': ebay_filters["categories"]["No Category"],  # Filter by category ID; "Cell Phones & Smartphones".
    '_sop': ebay_filters["sort_order"]["Time: newly listed"],  # Sort by "Time: newly listed"
    'LH_Sold': '1',  # Only sold listings (='1').
    'LH_Complete': '1',  # Only completed listings (='1').
    'LH_BIN': '0',  # Only Buy It Now listings (='1').
    'LH_Auction': '1',  # Only Auction Listings (='1').
    'LH_BO': '0',  # Only listings that accept offers (='1').
    'LH_FS': '0',  # Only Free Shipping listings (='1').
    '_ipg': '240',  # Number of items per page (='1'), Max is 240.
    'rt': 'nc'  # Result type; 'nc' indicates no cache to ensure the search results are fresh. 
    
}


In [6]:
# Create a prepared request to inspect the full URL
request  =requests.Request('GET', url, params=params)
prepared_request = request.prepare()
print(prepared_request.url)


https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone+13&LH_ItemCondition=3000&LH_PrefLoc=2&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=1&LH_Complete=1&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc


In [21]:
# Initialize variables
page_number = 0
items_list = []

# Loop over pages
while True:
    
    page_number += 1
    
    print(f'Scraping page: {page_number}')
          
    params['_pgn'] = page_number
    
    # Send GET request to eBay with the defined parameters
    response  = requests.get(url, params=params)
    html_content = response.text # Get the HTML content of the page
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
          
    # Find the next button
    next_button = soup.find('button', class_='pagination__next', type='next')

    # Check if the next button is disabled
    if next_button and next_button.get('aria-disabled') == 'true':
        print('No more pages pages to scrape')
        break
    else:
        # Extract items
        items = soup.find_all('div', class_='s-item__wrapper clearfix')

        # Extract Listings
        for item in items [2:]:
            title = item.find('div', class_='s-item__title').text
            price = item.find('span', class_='s-item__price').text
            link = item.find('a', class_='s-item__link')['href'].split('?')[0]
            image_url = item.find('div', class_='s-item__image-wrapper image-treatment').find('img').get('src','No image URL')

            # Define each item as a dictionary
            item_dict = {
                'Title': title,
                'Price': price,
                'Link': link,
                'Image Link': image_url
            }

            # Append the dictionary to the list
            items_list.append(item_dict)
            

Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
No more pages pages to scrape


In [22]:
len(items_list)

1680

In [23]:
# Turn the list into a Pandas DataFrame
items_df = pd.DataFrame(items_list)

In [24]:
items_df

,Title,Price,Link,Image Link
0,Apple iPhone 13 - 128GB Red - Unlocked-mint wi...,$356.14,https://www.ebay.com/itm/335543771393,https://i.ebayimg.com/images/g/zO8AAOSwpWBmlaI...
1,Apple iPhone 13 - 128 GB - Multicolor (T-Mobile),$290.00,https://www.ebay.com/itm/186647791239,https://i.ebayimg.com/images/g/JtEAAOSwBt9mx4k...
2,Apple iPhone 13 - 256 GB - Black (Unlocked),$390.00,https://www.ebay.com/itm/286014854123,https://i.ebayimg.com/images/g/8GQAAOSwrQlmv7y...
3,Pre-Owned - Apple iPhone 12 Pro Max 128GB Veri...,$283.00,https://www.ebay.com/itm/135196543757,https://i.ebayimg.com/images/g/RcMAAOSwOcBmox5...
4,Apple iPhone 13 - 128 GB - Black (Unlocked),$325.00,https://www.ebay.com/itm/405182489444,https://i.ebayimg.com/images/g/JfcAAOSwIolmxhT...
...,...,...,...,...
1675,Apple iPhone 13 - 128 GB (Unlocked) =Read Des...,$215.00,https://www.ebay.com/itm/166782870234,https://i.ebayimg.com/images/g/kk8AAOSwGERmUTO...
1676,UNLOCKED Apple iPhone 12 Pro 128GB Graphite 4G...,$281.00,https://www.ebay.com/itm/387023426864,https://i.ebayimg.com/images/g/fQAAAOSwSd9mTTE...
1677,Apple iPhone 13 A2482- 128 GB - Blue (Unlocke...,$357.00,https://www.ebay.com/itm/196408977114,https://i.ebayimg.com/images/g/7TcAAOSwirxmT-M...
1678,Apple iPhone 13 -128 GB - Alpine Green (Unlock...,$275.00,https://www.ebay.com/itm/315399799618,https://i.ebayimg.com/images/g/QKMAAOSwmp9mQrq...


In [27]:
# Define forbidden terms
forbidden_terms = [
    'refurbished',
    'iphone 12',
    'parts',
    'damaged',
    'locked',
    'pro',
    'mini',
    '256 gb',
    '512 gb',
    'verizon',
    'at&t', 
    't-mobile',
    'cricket',
    'metro',
    'boost',
    'read description'
]


In [28]:
# Create a boolean mask for filtering out forbidden terms
mask = ~items_df['Title'].str.lower().str.contains(r'\b(?:' + '|'.join(forbidden_terms) + r')\b')

# Apply the mask to filter the DataFrame
filtered_df = items_df[mask]


# Reset the index of the filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)

In [29]:
filtered_df

,Title,Price,Link,Image Link
0,Apple iPhone 13 - 128GB Red - Unlocked-mint wi...,$356.14,https://www.ebay.com/itm/335543771393,https://i.ebayimg.com/images/g/zO8AAOSwpWBmlaI...
1,Apple iPhone 13 - 128 GB - Black (Unlocked),$325.00,https://www.ebay.com/itm/405182489444,https://i.ebayimg.com/images/g/JfcAAOSwIolmxhT...
2,Apple iPhone 13 - 128 GB - Blue (Unlocked) No ...,$325.99,https://www.ebay.com/itm/326239794663,https://i.ebayimg.com/images/g/ovUAAOSwBERms91...
3,iPhone 13,$370.00,https://www.ebay.com/itm/335532956734,https://i.ebayimg.com/images/g/-5kAAOSwWaVmvtv...
4,Apple iPhone 13 - 128 GB - Blue (Unlocked) MLA...,$395.00,https://www.ebay.com/itm/315661520395,https://i.ebayimg.com/images/g/0BwAAOSwu4Rmu25...
...,...,...,...,...
635,Apple iPhone 13 A2482 128GB Network Unlocked -...,$324.00,https://www.ebay.com/itm/135077847513,https://i.ebayimg.com/images/g/W9sAAOSwlLNmVKZ...
636,Apple iPhone 13 - 128GB Midnight - Unlocked,$336.36,https://www.ebay.com/itm/375460154158,https://i.ebayimg.com/images/g/tZ0AAOSwikFmWer...
637,Apple iPhone 13 - 128GB - Blue (Unlocked),$309.98,https://www.ebay.com/itm/375460051022,https://i.ebayimg.com/images/g/fAAAAOSwEKdmWdN...
638,Apple iPhone 13 A2482- 128 GB - Blue (Unlocke...,$357.00,https://www.ebay.com/itm/196408977114,https://i.ebayimg.com/images/g/7TcAAOSwirxmT-M...


In [30]:
# Save the filtered DataFrame to a CSV file
filtered_df.to_csv('iphone 13 128.csv', index=False)